In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 15.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from nltk.stem import PorterStemmer
# from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Pemrograman Bahasa Alami/resesi_2023 (1).csv")
data

,Text,Label
0,Most of my business customers are people of we...,1
1,• Lomba Essay dan Infografis ACE 2023 “Inovasi...,1
2,"Check out my latest video ""Resesi Ekonomi 2023...",1
3,"@hariankompas 2023 ASU 🇺🇲 Resesi, 2024 ekonomi...",1
4,"Sri Mulyani: AS Batal Resesi, Dunia Selamat\n\...",1
...,...,...
127,Insecure Sama Otak Sndiri\nMau Learn Something...,-1
128,"Pagi2 kepikiran resesi 2023, krisis ekonomi ta...",-1
129,Duh! OJK Sebut Resesi Ekonomi Global Bisa Data...,-1
130,"Banyak yg bahas masalah perselingkuhan, gue ma...",-1


In [ ]:
Dup_Rows = data[data.duplicated()]
datafiks = data.drop_duplicates(keep='first')
datafiks

,Text,Label
0,Most of my business customers are people of we...,1
1,• Lomba Essay dan Infografis ACE 2023 “Inovasi...,1
2,"Check out my latest video ""Resesi Ekonomi 2023...",1
3,"@hariankompas 2023 ASU 🇺🇲 Resesi, 2024 ekonomi...",1
4,"Sri Mulyani: AS Batal Resesi, Dunia Selamat\n\...",1
...,...,...
127,Insecure Sama Otak Sndiri\nMau Learn Something...,-1
128,"Pagi2 kepikiran resesi 2023, krisis ekonomi ta...",-1
129,Duh! OJK Sebut Resesi Ekonomi Global Bisa Data...,-1
130,"Banyak yg bahas masalah perselingkuhan, gue ma...",-1


In [ ]:
#Remove Puncutuation
preprocessing = []
clean_tag = re.compile('@\S+')
clean_url = re.compile('https?:\/\/.*[\r\n]*')
clean_hastag = re.compile('#\S+')
clean_symbol = re.compile('[^0-9a-zA-Z]')
def clean_punct(text):
    text = clean_tag.sub('', text)
    text = clean_url.sub('', text)
    text = clean_hastag.sub(' ', text)
    text = clean_symbol.sub(' ', text)
    return text
# Buat kolom tambahan untuk data description yang telah diremovepunctuation   
preprocessing = datafiks['Text'].apply(clean_punct)
clean=pd.DataFrame(preprocessing)
clean

,Text
0,Most of my business customers are people of we...
1,Lomba Essay dan Infografis ACE 2023 Inovasi...
2,Check out my latest video Resesi Ekonomi 2023...
3,2023 ASU Resesi 2024 ekonomi dunia kolaps...
4,Sri Mulyani AS Batal Resesi Dunia Selamat M...
...,...
127,Insecure Sama Otak Sndiri Mau Learn Something ...
128,Pagi2 kepikiran resesi 2023 krisis ekonomi ta...
129,Duh OJK Sebut Resesi Ekonomi Global Bisa Data...
130,Banyak yg bahas masalah perselingkuhan gue ma...


In [ ]:
# def clean_lower(lwr):
#     lwr = lwr.lower() # lowercase text
#     return lwr
# Buat kolom tambahan untuk data description yang telah dicasefolding  
casefolding = clean['Text'].str.lower()
casefolding

0      most of my business customers are people of we...
1        lomba essay dan infografis ace 2023  inovasi...
2      check out my latest video  resesi ekonomi 2023...
3       2023 asu    resesi  2024 ekonomi dunia kolaps...
4      sri mulyani  as batal resesi  dunia selamat  m...
                             ...                        
127    insecure sama otak sndiri mau learn something ...
128    pagi2 kepikiran resesi 2023  krisis ekonomi ta...
129    duh  ojk sebut resesi ekonomi global bisa data...
130    banyak yg bahas masalah perselingkuhan  gue ma...
131      indonesia paling parah kena resesi ekonomi 2023
Name: Text, Length: 131, dtype: object

In [ ]:
# Membuat kamus slang words dan kata Indonesia yang benar
slang_dict = pd.read_csv("https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt", sep=" ", header=None)

# Membuat fungsi untuk mengubah slang words menjadi kata Indonesia yang benar
def replace_slang_words(text):
    words = nltk.word_tokenize(text.lower())
    words_filtered = [word for word in words if word not in stopwords.words('indonesian')]
    for i in range(len(words_filtered)):
        if words_filtered[i] in slang_dict:
            words_filtered[i] = slang_dict[words_filtered[i]]
    return ' '.join(words_filtered)

# penggunaan
# casefolding=[]
# for i in range(len(preprocessing)):
#   casefolding.append(preprocessing[i])

slang_words=[]
for i in range(len(casefolding)):
  slang = replace_slang_words(casefolding[i])
  slang_words.append(slang)

slang = pd.DataFrame(slang_words, columns=["Token"])
slang

In [ ]:
!pip install indonlp==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from indoNLP.preprocessing import replace_slang
from indoNLP.preprocessing import replace_word_elongation
replace_slang
# replace internet slang 
def slang(text):
    y = replace_slang(text)
    # hasil = replace_word_elongation(text)
    return y
# Buat kolom tambahan untuk data description yang telah dicasefolding  
slang = casefolding.apply(slang)
slang

NameError: ignored

In [ ]:
from nltk.tokenize import word_tokenize
def tokenize(review):
    token = word_tokenize(review)
    return token
# Buat kolom tambahan untuk data description yang telah dicasefolding  
token = slang.apply(tokenize)
token

0      [most, of, my, business, customers, are, peopl...
1      [lomba, essay, dan, infografis, ace, 2023, ino...
2      [check, out, my, latest, video, resesi, ekonom...
3      [2023, asu, resesi, 2024, ekonomi, dunia, kola...
4      [sri, mulyani, as, batal, resesi, dunia, selam...
                             ...                        
127    [insecure, sama, otak, sendiri, mau, learn, so...
128    [pagi2, kepikiran, resesi, 2023, krisis, ekono...
129    [duh, ojk, sebut, resesi, ekonomi, global, bis...
130    [banyak, yang, bahas, masalah, perselingkuhan,...
131    [indonesia, paling, parah, kena, resesi, ekono...
Name: Text, Length: 131, dtype: object

In [ ]:
listStopword =  set(stopwords.words('indonesian'))
listStopword2 =  set(stopwords.words('english'))
def stopword(text):
  removed=[]
  for t in text:
    if t not in listStopword:
      if t not in listStopword2:
        removed.append(t)
  return removed

# Buat kolom tambahan untuk data description yang telah dicasefolding  
stopword=token.apply(stopword)
stopword

0      [business, customers, people, western, europe,...
1      [lomba, essay, infografis, ace, 2023, inovasi,...
2      [check, latest, video, resesi, ekonomi, 2023, ...
3      [2023, asu, resesi, 2024, ekonomi, dunia, kola...
4      [sri, mulyani, batal, resesi, dunia, selamat, ...
                             ...                        
127    [insecure, otak, learn, something, tau, cocokn...
128    [pagi2, kepikiran, resesi, 2023, krisis, ekono...
129     [duh, ojk, resesi, ekonomi, global, cepat, 2023]
130    [bahas, perselingkuhan, gue, kepikiran, resesi...
131      [indonesia, parah, kena, resesi, ekonomi, 2023]
Name: Text, Length: 131, dtype: object

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemming process
def stemming(text):
  hasil=[]
  for k in text:
    hasil.append(stemmer.stem(k))
  return hasil

# Buat kolom tambahan untuk data description yang telah dicasefolding  

stemming=stopword.apply(stemming)
# stemming = pd.DataFrame(stemming, columns=["Text"])
gabung=[]
for i in range(len(stemming)):
  joinkata = ' '.join(stemming[i])
  gabung.append(joinkata)

result = pd.DataFrame(gabung, columns=['Join Kata'])
result

KeyError: ignored